# Лабораторная работа 7. Управление ресурсами в однопроцессорной системе с неоднородными заявками
Выполнил: Пакало Александр Сергеевич, студент РТ5-81Б

## Задание 1
В однопроцессорную систему случайным образом поступают на выполнение $m$
разных типов программ, отличающихся известной трудоемкостью
$Q_1, Q_2,\dots, Q_m$.
Входящий поток простейший с интенсивностью $\lambda$.

Представив данную систему как одноканальную СМО с неограниченной очередью,
вычислить среднее время обслуживания программ, считая длительность
обслуживания случайной величиной (теоретически и экспериментально).

Реализовать алгоритм SPT, выбирая из очереди заявки в соответствии с их
приоритетом по трудоемкости. Рассчитать среднее время обслуживания программ.
Сравнить полученные результаты.

Реализовать алгоритм RR при заданном кванте времени $q$. Оценить среднее время
обслуживания программ. Сравнить полученные результаты. Выяснить, как влияет
величина кванта на среднее время обслуживания программ.

In [1]:
Variant <- 5
set.seed(Variant)
m <- sample(c(6:20), 1)
lambda <- runif(1, 0.1, 2)
Q <- rexp(m, 0.3)
q <- sample(c(1:4), 1)
View(data.frame(m, q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
7,1,1.401915


[1] 0.2417931 1.3403831 0.1863999 2.0529013 0.2552007 2.9172096 6.6709868


Заведем таблицу результатов

In [59]:
results <- data.frame(0, 0, 0, 0, 0)
colnames(results) <- c("M/M/1/infty theoretical", "M/M/1/infty practical", "SPT theoretical", "SPT practical", "Round Robin")

### СМО вида $М/М/1/\infty$
Представим данную систему как одноканальную СМО с неограниченной очередью.

#### Теоретически

In [72]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [73]:
y <- lambda / mu
y

[1] 2.736714

Так как $y > 1$, поменяем $\lambda$.

In [75]:
lambda <- 0.3

In [76]:
t2 <- mean(Q)
mu <- 1 / t2
mu

[1] 0.5122623

In [77]:
y <- lambda / mu
y

[1] 0.5856375

In [78]:
results[1] <- 1 / mu / (1 - y)
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,3.662282,4.008172,2.5725,2.5725


#### Численно

In [111]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [112]:
programs <- trajectory("programs' path") %>%
    seize("server", amount = 1) %>%
    timeout(function() rexp(1, mu)) %>%
    release("server", amount = 1)

In [113]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 10000 | next: 10001.2542877312
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 1(1) | queue status: 0(Inf) }
{ Source: programs | monitored: 1 | n_generated: 2942 }

In [115]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,4.263362,5.042783,0.779421017,TRUE,1
programs1,6.746470,7.256152,0.509681929,TRUE,1
programs2,7.619343,7.906546,0.287202954,TRUE,1
programs3,8.957836,11.773471,2.815635448,TRUE,1
programs4,9.077252,11.812946,0.039474524,TRUE,1
programs5,9.511587,13.207806,1.394859912,TRUE,1
programs6,10.964045,13.508030,0.300223750,TRUE,1
programs7,11.346586,14.121309,0.613279763,TRUE,1
programs8,15.411978,15.416274,0.004295376,TRUE,1


In [116]:
results[2] <- mean(arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5725,2.5725


### Алгоритм SPT

#### Теоретически
Среднее время обслуживания складывается из ожидания в очереди и времени
выполнения, усредненным по всем заявкам:
$$
T_{\text{сист}} = \frac{1}{m} \left( Q'_1 + \left( Q'_1 + Q'_2 \right) +
\left(Q'_1 + Q'_2 + Q'_3 \right) + \dots + \sum^m_{i = 1}Q'_i \right)
$$
где $Q'_i$ - $i$-й элемент массива $Q'$,
**отсортированного по возрастанию** массива Q.

In [117]:
Q_sorted <- sort(Q)
Q_sorted

[1] 0.1863999 0.2417931 0.2552007 1.3403831 2.0529013 2.9172096 6.6709868

Суммы первых i элементов:

In [118]:
Q_progression_sums <- lapply(
    seq_along(Q_sorted),
    function(i) sum(head(Q_sorted, i))
)
Q_progression_sums

[[1]]
[1] 0.1863999

[[2]]
[1] 0.428193

[[3]]
[1] 0.6833937

[[4]]
[1] 2.023777

[[5]]
[1] 4.076678

[[6]]
[1] 6.993888

[[7]]
[1] 13.66487

Итоговая сумма

In [4]:
sum_of_Q_progression_sums <- sum(unlist(Q_progression_sums))
sum_of_Q_progression_sums

[1] 28.0572

In [119]:
results[3] <- 1 / m * sum_of_Q_progression_sums
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5725,2.5725


#### Численно
Реализуем SPT с помощью simmer, воспользовавшись механизмом select,
выбирающим из очереди значение по определенной стратегии.

In [219]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

spt.env <- simmer("SuperDuperSptSim")
spt.env

simmer environment: SuperDuperSptSim | now: 0 | next: 
{ Monitor: in memory }

Добавим $m$ генераторов. Каждый будет иметь приоритет в зависимости от
скорости выполнения программы. Генератор, создающий программы с наибольшей
длительностью выполнения, будет иметь наименьший приоритет.

In [220]:
SIMULATION_TIME <- 10000

spt.env %>%
    add_resource("server", 1)

Q_sorted_decr <- sort(Q, decreasing = TRUE)

spt.programs_trajectory <- function(time_to_execute) {
    return(
        trajectory("programs' path") %>%
            seize("server", 1) %>%
            timeout(function() rexp(1, 1 / time_to_execute)) %>%
            release("server", 1)
    )
}

for (Q_i in seq_along(Q_sorted_decr)) {
    priority <- Q_i

    name <- paste0("programs", Q_i)

    spt.env %>% add_generator(
        name,
        spt.programs_trajectory(Q_sorted_decr[Q_i]),
        priority = priority,
        preemptible = priority,
        distribution = function() rexp(1, lambda / m)
    )
}

spt.env %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSptSim | now: 0 | next: 
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }

simmer environment: SuperDuperSptSim | now: 10000 | next: 10001.7134950434
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs1 | monitored: 1 | n_generated: 424 }
{ Source: programs2 | monitored: 1 | n_generated: 447 }
{ Source: programs3 | monitored: 1 | n_generated: 445 }
{ Source: programs4 | monitored: 1 | n_generated: 431 }
{ Source: programs5 | monitored: 1 | n_generated: 423 }
{ Source: programs6 | monitored: 1 | n_generated: 445 }
{ Source: programs7 | monitored: 1 | n_generated: 398 }

In [221]:
spt.arrivals <- spt.env %>%
    get_mon_arrivals()
spt.arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs60,8.963183,9.331645,0.36846195,TRUE,1
programs30,12.968233,13.187424,0.21919104,TRUE,1
programs61,13.436932,13.936521,0.49958874,TRUE,1
programs20,14.372574,15.095163,0.72258944,TRUE,1
programs62,18.145967,18.282460,0.13649244,TRUE,1
programs10,19.990619,20.031901,0.04128154,TRUE,1
programs21,21.401813,21.556375,0.15456209,TRUE,1
programs50,39.493284,39.592314,0.09902948,TRUE,1
programs22,47.252065,47.503320,0.25125458,TRUE,1


In [222]:
results[5] <- mean(spt.arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5598,0.1931421


### Алгоритм Round Robin
Реализуем Round Robin с помощью simmer, воспользовавшись механизмом select,
выбирающим из очереди значение по определенной стратегии.

In [170]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [171]:
programs <- trajectory("programs' path") %>%
    select("server", "round-robin") %>%
    seize_selected(1) %>%
    timeout(function() rexp(1, mu)) %>%
    release_selected(1)

In [172]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", 1) %>%
    add_generator("programs", programs, function() rexp(1, lambda)) %>%
    run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 10000 | next: 10000.6880190946
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programs | monitored: 1 | n_generated: 2958 }

In [173]:
arrivals <- env %>%
    get_mon_arrivals()
arrivals

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programs0,11.71948,11.72665,0.007171833,TRUE,1
programs1,11.97214,16.47420,4.502058210,TRUE,1
programs2,13.18187,19.27042,2.796220815,TRUE,1
programs3,13.84978,19.41637,0.145955088,TRUE,1
programs4,17.25988,19.73762,0.321247764,TRUE,1
programs5,17.36269,21.07808,1.340459151,TRUE,1
programs6,18.08577,21.35824,0.280164119,TRUE,1
programs7,32.47119,33.76257,1.291385800,TRUE,1
programs8,34.91324,35.59000,0.676756013,TRUE,1


In [185]:
results[5] <- mean(arrivals %>% with(end_time - start_time))
results

M/M/1/infty theoretical,M/M/1/infty practical,SPT theoretical,SPT practical,Round Robin
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.711152,4.545182,4.008172,2.5598,4.91254


Как видно, практические вычисления совпадают теоретическими с некоторой
погрешностью, которая уменьшается при увеличении числа экспериментов.

При этом система, выполненная с помощью алгоритма Round Robin оказалась
быстрее обычной системы, а система, реализованная с алгоритмом SPT - самой
быстрой.